In [233]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel

model = LocalEmbeddingModel(
    model_name='allenai/specter2_base',  # 'sentence-transformers/all-MiniLM-L6-v2',
    chunk_size=512,
    chunk_overlap=64,
    batch_size=8
)

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/717k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/754 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Using device: cpu


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [234]:
import torch

tmp_input = "Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model. You can concatenate multiple poolings together."

inputs = [tmp_input*10, tmp_input*3, tmp_input, tmp_input*30]

_, all_chunks_encoded = model.chunk_and_encode(inputs, progress_bar=True)

# Flatten encoded
tmp = {}
for single_text_encoded in all_chunks_encoded:
    for k, v in single_text_encoded.items():
        if k not in tmp:
            tmp[k] = []
        tmp[k].append(v)
encoded_flattened = {k: torch.cat(v) for k, v in tmp.items()}

# Get embeddings for all chunks
print(f"Generating embeddings for {len(list(encoded_flattened.values())[0])} chunks...")
embeddings = model.get_embeddings(encoded_flattened, progress_bar=True)

Chunking and encoding: 100%|██████████| 4/4 [00:00<00:00, 585.61it/s]


Generating embeddings for 8 chunks...


Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [231]:
from torch.nn import functional as F

_, encoded = model.chunk_and_encode(tmp_input*100)
emb = model.get_embeddings(encoded)


encoded_input = model.tokenizer(tmp_input*100, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model.model(**encoded_input)

# Perform pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)



In [232]:
emb[0] == sentence_embeddings[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

## Dev for new cleaner store setup

In [24]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store import LocalEmbeddingModel, FAISSDocumentStore
from tqdm import tqdm

In [25]:
ds = FAISSDocumentStore(db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/allenai_specter2_base_test2')
assert ds.load_store()

Using device: cpu
Loaded index with 103 vectors


In [30]:
ds.search('What is the main idea of the paper?', top_k=10)

[{'rank': 1,
  'score': 0.532930805335529,
  'doc_id': 'W2772709170',
  'doi': '10.48550/arxiv.1712.01815',
  'topic': 'Artificial Intelligence in Games'},
 {'rank': 2,
  'score': 0.5323105859180169,
  'doc_id': 'W4310561607',
  'doi': '10.48550/arxiv.2211.17111',
  'topic': 'Parallel Computing and Optimization Techniques'},
 {'rank': 3,
  'score': 0.5287573212579326,
  'doc_id': 'W2222512263',
  'doi': '10.1002/nav.3800020109',
  'topic': 'Optimization and Mathematical Programming'},
 {'rank': 4,
  'score': 0.526998484957759,
  'doc_id': 'W2755088640',
  'doi': 'nan',
  'topic': 'Complex Network Analysis Techniques'},
 {'rank': 5,
  'score': 0.5267845637437224,
  'doc_id': 'W2158131535',
  'doi': 'nan',
  'topic': 'Machine Learning and Algorithms'},
 {'rank': 6,
  'score': 0.5267383869762133,
  'doc_id': 'W2151554678',
  'doi': '10.1109/4235.585893',
  'topic': 'Metaheuristic Optimization Algorithms Research'},
 {'rank': 7,
  'score': 0.5255418443014074,
  'doc_id': 'W2964081807',
  '

In [22]:
import pandas as pd

a = {
    'chunk_id': ds.chunk_store['chunk_id'].tolist(),
    'doc_id': ds.chunk_store['doc_id'].tolist(),
    'text': ds.chunk_store['text'].tolist(),
}

chunk_store = pd.DataFrame(a).astype({'chunk_id': 'int64'}).set_index('chunk_id')

In [23]:
chunk_store

,doc_id,text
chunk_id,,
0,W2194775991,deeper neural networks are more difficult to t...
1,W1686810756,in this work we investigate the effect of the ...
2,W4385245566,the dominant sequence transduction models are ...
3,W2108598243,the explosion of image data on the internet ha...
4,W2133665775,objective methods for assessing perceptual ima...
...,...,...
1011,W2013439747,this paper discusses the experimental setup of...
1012,W3158642773,abstract despite the rapid development and pro...
1013,W3005695990,density estimation aims to predict the spatial...
